In [ ]:
import cdsw
import os
import cmlapi
import json, requests
import time

In [ ]:
#project_name = 'olivers llm project'
project_name ='BBH-LLM-POV'
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))

In [ ]:
client.list_project_names()

In [ ]:
proj_id = client.list_projects()

In [ ]:
#proj_id

In [ ]:
proj_id = client.list_projects(search_filter=json.dumps({"name":project_name }))

In [ ]:
#proj_id

In [ ]:
proj_id.projects[0].id

In [ ]:
proj_id = client.list_projects(search_filter=json.dumps({"name":project_name })).projects[0].id

#### Running Chroma Job

In [ ]:
chroma_job_name = 'Populate Chroma Vector DB'
chroma_target_job = client.list_jobs(proj_id, search_filter=json.dumps({"name": chroma_job_name}))

In [ ]:
job_run = client.create_job_run(cmlapi.CreateJobRunRequest(),project_id = proj_id, job_id = chroma_target_job.jobs[0].id)

#### Running Populate Pinecone DB job

In [ ]:
pinecone_job_name= 'populate pinecone'
pinecone_target_job = client.list_jobs(proj_id, search_filter=json.dumps({"name": pinecone_job_name}))

In [ ]:
job_run = client.create_job_run(cmlapi.CreateJobRunRequest(),project_id = proj_id, job_id = pinecone_target_job.jobs[0].id)

#### List Job Runs

#### Run pdf reader then db loads

In [ ]:
pdf_reader_job_name = 'pdf_reader'
pdf_reader_target_job = client.list_jobs(proj_id, search_filter=json.dumps({"name": pdf_reader_job_name}))

In [ ]:
job_run = client.create_job_run(cmlapi.CreateJobRunRequest(),project_id = proj_id, job_id = pdf_reader_target_job.jobs[0].id)

In [ ]:
listed_jobs = client.list_job_runs(project_id=proj_id, job_id = pdf_reader_target_job.jobs[0].id)

In [ ]:
job_status = ''
while job_status != 'ENGINE_SUCCEEDED' and job_status != 'ENGINE_FAILED' :
    listed_jobs = client.list_job_runs(project_id=proj_id, job_id = pdf_reader_target_job.jobs[0].id)
    job_status = listed_jobs.job_runs[-1].status
    print('job status is', job_status)
    time.sleep(5)
print('job done')

#### Try jobs in succession

In [ ]:
job_chain = ['pdf_reader', 'Populate Chroma Vector DB', 'populate pinecone']
    
for job in job_chain:
    print(f'running {job}')
    if job == 'pdf_reader':
        target_job = client.list_jobs(proj_id, search_filter=json.dumps({"name": job}))
        print('job running is', job)
        print('target_job is',target_job)
        print('target_job.jobs',target_job.jobs)
        print('type of target_job.jobs',type(target_job.jobs))
        job_run = client.create_job_run(cmlapi.CreateJobRunRequest(),project_id = proj_id, job_id = target_job.jobs[0].id)
        job_status = ''

        
        while job_status != 'ENGINE_SUCCEEDED':
            # api forcing 50 job runs to show up per page. 
            # after 50 job runs, this will not work
            listed_jobs = client.list_job_runs(project_id=proj_id, job_id=target_job.jobs[0].id, page_size=50)
            job_status = listed_jobs.job_runs[-1].status

            if job_status == 'ENGINE_FAILED':
                print(f'{job} has failed.')
                raise RuntimeError(f"Job {job} failed. Halting the job chain.")
                
            # Update the user that the job is still processing
            #result_messages.append(f"{job} is still processing... Please wait.")
            time.sleep(5)
    else:
        target_job = client.list_jobs(proj_id, search_filter=json.dumps({"name": job}))
        job_run = client.create_job_run(cmlapi.CreateJobRunRequest(),project_id = proj_id, job_id = target_job.jobs[0].id)

    print(f'{job} is completed')

In [ ]:
# job_chain = ['pdf_reader','Populate Chroma Vector DB','populate pinecone']

# for job in job_chain:
#     print(f'running {job}')
#     if job == 'pdf_reader':
#         target_job = client.list_jobs(proj_id, search_filter=json.dumps({"name": job}))
#         job_run = client.create_job_run(cmlapi.CreateJobRunRequest(),project_id = proj_id, job_id = target_job.jobs[0].id)
#         job_status = ''
#         while job_status != 'ENGINE_SUCCEEDED':
#             listed_jobs = client.list_job_runs(project_id=proj_id, job_id=target_job.jobs[0].id)
#             job_status = listed_jobs.job_runs[-1].status
            
#             if job_status == 'ENGINE_FAILED':
#                 print(f'{job} has failed.')
#                 raise RuntimeError(f"Job {job} failed. Halting the job chain.")
            
#             time.sleep(5)
#     else:
#         target_job = client.list_jobs(proj_id, search_filter=json.dumps({"name": job}))
#         job_run = client.create_job_run(cmlapi.CreateJobRunRequest(),project_id = proj_id, job_id = target_job.jobs[0].id)

#     print(f'{job} is completed')

In [ ]:
job_chain = ['pdf_reader','Populate Chroma Vector DB','populate pinecone']

In [ ]:
job_chain[0]

In [ ]:
target_job = client.list_jobs(proj_id, search_filter=json.dumps({"name": job_chain[0]}))

In [ ]:
listed_jobs_runs = client.list_job_runs(project_id=proj_id, job_id=target_job.jobs[0].id, page_size=50)

In [ ]:
listed_jobs_runs.job_runs[-1]

In [ ]:
len(listed_jobs.job_runs)